In [27]:
import numpy as np
import math
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
import os
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")
os.chdir("/Users/nsy/Documents/Grad2/5671/HW4")
insurance_df = pd.read_csv("training_data.csv",header =0)
insurance_df.head()

,STATE,REGION,SEX,PROFESSION,AGE,HAS_CHILDREN,SALARY,N_OF_DEPENDENTS,CAR_OWNERSHIP,HOUSE_OWNERSHIP,...,MORTGAGE_AMOUNT,N_TRANS_ATM,N_MORTGAGES,N_TRANS_TELLER,CREDIT_CARD_LIMITS,N_TRANS_KIOSK,N_TRANS_WEB_BANK,LTV,LTV_BIN,BUY_INSURANCE
0,CA,West,M,IT Staff,63,0,59461,0,1,0,...,0,0,0,0,900,2,0,22165.25,HIGH,0
1,CA,West,M,PROF-9,36,1,60271,1,1,1,...,3063,5,1,1,900,0,3063,22167.75,HIGH,1
2,FL,South,M,Programmer/Developer,21,0,64738,3,1,0,...,0,2,0,2,1000,4,0,13784.50,LOW,1
3,WA,West,F,IT Staff,47,1,65071,1,1,1,...,15000,6,1,7,500,3,3000,24467.75,HIGH,1
4,UT,Southwest,M,Technical Writer,26,0,61674,3,1,1,...,300,0,1,0,1500,5,300,23518.50,HIGH,0


In [29]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [30]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np

# target variable distribution
insurance_df.groupby(['BUY_INSURANCE'])[['BUY_INSURANCE']].count()

,BUY_INSURANCE
BUY_INSURANCE,
0,529
1,181


In [31]:
cat_vars=['STATE','REGION','SEX','PROFESSION','MARITAL_STATUS']
cat_df = pd.get_dummies(insurance_df[cat_vars], prefix='var')
insurance_df = pd.concat([insurance_df,cat_df],axis = 1)

In [32]:
data_vars = insurance_df.columns.values.tolist()
selected_columns = list(set(data_vars).difference(set(cat_vars)))
# np.array(selected_columns)

# now only get variables after converting to dummy variables
data = insurance_df[selected_columns]
data.head()

,var_PROF-52,var_South,var_IT Staff,var_PROF-46,T_AMOUNT_AUTOM_PAYMENTS,var_Accountant,HAS_CHILDREN,var_Not specified,var_PROF-51,var_PROF-55,...,var_PROF-34,var_PROF-29,var_Programmer/Developer,var_PROF-61,MONEY_MONTLY_OVERDRAWN,var_WIDOWED,var_PROF-14,var_PROF-44,var_Cashier,var_Professor
0,0,0,1,0,504,0,0,0,0,0,...,0,0,0,0,53.06,0,0,0,0,0
1,0,0,0,0,1311,0,1,0,0,0,...,0,0,0,0,54.51,0,0,0,0,0
2,0,1,0,0,539,0,0,0,0,0,...,0,0,1,0,53.02,0,0,0,0,0
3,0,0,1,0,59437,0,1,0,0,0,...,0,0,0,0,57.70,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,53.06,0,0,0,0,0


In [33]:
relevant_features = []
for col in data.columns:
    thres = 0.1
    try:
        corr = data['BUY_INSURANCE'].corr(data[col])
    except:
        print(col)
        
    if abs(corr) > thres:
        relevant_features.append(col)
    
relevant_features

LTV_BIN


['var_SINGLE',
 'var_NC',
 'BANK_FUNDS',
 'var_DIVORCED',
 'BUY_INSURANCE',
 'N_OF_DEPENDENTS',
 'MONTHLY_CHECKS_WRITTEN',
 'N_TRANS_TELLER',
 'N_TRANS_ATM',
 'var_PROF-8',
 'var_Nurse',
 'var_F',
 'var_M',
 'CHECKING_AMOUNT',
 'var_Programmer/Developer',
 'MONEY_MONTLY_OVERDRAWN']

In [34]:
data = data[relevant_features]

In [36]:
from sklearn.model_selection import train_test_split
X = data[data.columns.difference(['BUY_INSURANCE','LTV_BIN'])]

In [37]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = data[['BUY_INSURANCE']].values.astype('float64')

In [45]:
!pip install imblearn 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X_train, y_train)
X_resampled, y_resampled = shuffle(X_resampled, y_resampled)

     |████████████████████████████████| 174kB 1.4MB/s eta 0:00:01


Using TensorFlow backend.


In [46]:
y_train_df= pd.DataFrame(data=y_resampled,columns=['BUY_INSURANCE'])
y_train_df.groupby(['BUY_INSURANCE'])[['BUY_INSURANCE']].count()

,BUY_INSURANCE
BUY_INSURANCE,
0.0,118
1.0,118


In [68]:
import numpy as np
import math
from sklearn import preprocessing
# Should use the same training/test datasets above: X_resampled, y_resampled, X_test_os, y_test_os

# function that normalizes each predictor
def normalize(X):
    Xnorm = (X-np.mean(X))/np.std(X)
    return Xnorm

# function that return sigmoid of z
def sigmoid(z):
    g = 1/(1 + np.exp(-z))
    return g

def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))


def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

def computeCost(paras, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    return J

def computeGrad(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  
    delta2 = np.zeros(theta2.shape)  
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  
        z2t = z2[t,:]  
        a2t = a2[t,:]  
        ht = h[t,:]  
        yt = y[t,:]  
        
        d3t = ht - yt  
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return grad

def optimize(paras,X,y,maxiter, lambd=0.01):
    cost=[]
    for i in range(maxiter):
        a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
        cost=computeCost(paras, input_size, hidden_size, num_labels, X, y, learning_rate)
        costs.append(cost)
        grad=computeGrad(params, input_size, hidden_size, num_labels, X, y, learning_rate)
        parameters=gradient(paras,grad,learning_rate)
    return parameters

def predict(parameters,X):
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    prediction = (h>0.5).astype(int)
    return prediction

In [ ]:
from sklearn import metrics
X = normalize(X_train.T)
y = y_train.T
input_size = X.shape[0]
hidden_size = input_size+1
num_labels = y.shape[0]
m = X.shape[0]
learning_rate = 1
theta1 = np.random.randn(hidden_size, 1 + input_size) * 0.01
theta2 = np.random.randn(1, 1 + hidden_size) * 0.01
paras = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25
theta = optimize(paras, X_resampled, y_resampled, 10000, 0.01)
pred = predict(parameters, X_resampled)

In [ ]:
predict(parameters,X_resampled)